In [ ]:
import torch


class Rnn_layer:
    def __init__(self,vocab_size,hidden_size):
        self.wh=torch.randn(hidden_size,hidden_size,requires_grad=True)
        self.wx=torch.randn(vocab_size,hidden_size,requires_grad=True)
        self.wy=torch.rand(hidden_size,vocab_size)
        self.by=torch.randn(1,vocab_size)
        self.bh=(1,hidden_size)
        self.ht_1=torch.zeros(1,hidden_size)
    def hiden_satate(self,xt):
        hiden_state= self.wh*self.ht_1+xt*self.wx+self.bh
        self.ht_1=hiden_state
        return hiden_state
    def output(self,hiden_state):
        return hiden_state*self.wy+self.by
    

In [ ]:

class genrative_model:
    def __init__(self):
        self.layers=[]
        self.criterion=torch.nn.CrossEntropyLoss()
        self.optimizer=torch.optim.Adam()

    def add_layer(self,vocab_size,hidden_size):
        self.layers.append(
            Rnn_layer(vocab_size,hidden_size)
        )
    def train(self,len_seq,epochs,xtrain,ytrain):
        for epoch in range(epochs):
         loss=0
         for t in len(len_seq):
            xt=xtrain[t]
            for layer in self.layers:
               ht=layer.hiden_state(xt)
               xt=ht
            ypred=layer.output(xt)
            l=self.criterion(ypred,ytrain[t])
            with torch.no_grad():
               l.backward()
               self.optimizer.step()
            loss+=l
         print(f"{epoch} loss :{loss/len(len_seq)}")


In [ ]:
class classification_model:
    def __init__(self):
        self.layers=[]
        self.criterion=torch.nn.CrossEntropyLoss()
        self.optimizer=torch.optim.Adam()
    def add_layer(self,vocab_size,hidden_size):
        self.layers.append(
            Rnn_layer(vocab_size,hidden_size)
        )
    def train(self,len_seq,epochs,xtrain,ytrain):
        for epoch in range(epochs):
         ypred=[]
         for t in len(len_seq):
            xt=xtrain[t]
            for layer in self.layers:
               ht=layer.hiden_state(xt)
               xt=ht
            ypred.append(layer.output(xt))
         loss=self.criterion(torch.tensor(ypred),ytrain)
         with torch.no_grad():
               loss.backward()
               self.optimizer.step()        
        print(f"{epoch} loss :{loss}")
    

        
  


In [ ]:
class LStm(Rnn_layer):
    def __init__(self, vocab_size, hidden_size):
        super().__init__( vocab_size, hidden_size)
        self.wh_i=torch.randn(hidden_size,hidden_size)
        self.wx_i=torch.randn(vocab_size,hidden_size)
        self.wh_f=torch.randn(hidden_size,hidden_size)
        self.wx_f=torch.randn(vocab_size,hidden_size)
        self.wh_c=torch.randn(hidden_size,hidden_size)
        self.wx_c=torch.randn(vocab_size,hidden_size)
        self.wh_o=torch.randn(hidden_size,hidden_size)
        self.wx_o=torch.randn(vocab_size,hidden_size)
        self.bi=torch.randn(1,hidden_size)
        self.bf=torch.randn(1,hidden_size)
        self.bo=torch.randn(1,hidden_size)
        self.bh=torch.randn(1,hidden_size)


     # input gate
    def input_gate(self, xt, ht_1):
        return torch.sigmoid(
            xt @ self.wxi + ht_1 @ self.whi + self.bi
        )

    # forget gate
    def forget_gate(self, xt, ht_1):
        return torch.sigmoid(
            xt @ self.wxf + ht_1 @ self.whf + self.bf
        )

    # output gate
    def output_gate(self, xt, ht_1):
        return torch.sigmoid(
            xt @ self.wxo + ht_1 @ self.who + self.bo
        )

    # candidate cell state
    def candidate_info(self, xt, ht_1):
        return torch.tanh(
            xt @ self.wxc + ht_1 @ self.whc + self.bc
        )

    # memory (cell state) update
    def memory_update(self, ft, it, ct_bar, ct_1):
        return ft * ct_1 + it * ct_bar

    # hidden state update
    def hidden_state(self, ot, ct):
        return ot * torch.tanh(ct)

    


    
    

In [ ]:
class Gru(Rnn_layer):
    def __init__(self, vocab_size, hidden_size):
        super().__init__( vocab_size, hidden_size)
        self.Wrx=torch.randn(vocab_size,hidden_size)
        self.Ur=torch.randn(hidden_size,hidden_size)
        self.Wrz=torch.randn(hidden_size,hidden_size)
        self.Uz=torch.randn(vocab_size,hidden_size)
        self.wx_h=torch.randn(vocab_size,hidden_size)
        self.wh_h=torch.randn(vocab_size,hidden_size)
        self.br=torch.randn(1,hidden_size)
        self.bz=torch.randn(1,hidden_size)
        self.bh=torch.randn(1,hidden_size)
    def reset_gate(self,xt,ht_1):
        return torch.sigmoid(
            xt*self.Wrx+ht_1*self.Ur+self.br

        )
    def update(self,xt,ht_1):
        return torch.sigmoid(
            xt*self.Wrz+ht_1*self.Uz+self.bz
        )
    def candidate_state(self,xt,rt,ht_1,):
        return torch.tanh(
            xt*self.wx_h+(rt*ht_1)*self.wh_h+self.bh
        )
    def new_state(self,zt,ht_1,candidate):
        zt*ht_1+(1-zt)*candidate
        
    
